
Given:

* a Markov model with parameters $\lambda = \{ \mathbf{a},\mathbf{A},\mathcal{B} \}$
* an observation $\mathbf{o} \in \mathcal{R}^T$ where $T$ are the time steps 

Find the most probable state sequence: $\mathbf{s}^\star$ 
     
Note: see ForwardBackward for detailed definitions of the variables above.

Again as a baseline we check a brute force solution, that is compute $L_{\lambda}( \mathbf{o} | \mathbf{s} )$ for all $\mathbf{s} \in \mathcal{S}$:

$l = \arg \max \{ L_\lambda (\mathbf{o} | \mathbf{s}) : \mathbf{s} \in \mathcal{S} \}$

where $\mathbf{v} = \mathbf{s}_l$ will be the most probable state sequence.

Using Bayes:

$L_\lambda (\mathbf{o} | \mathbf{s}) = L(\mathbf{s} | \mathbf{o} ) P(\mathbf{s}) / P( \mathbf{o} )$

we drop the denominator $P(\mathbf{o})$ and we know that:

  * $L_{\lambda}(\mathbf{s} | \mathbf{o} ) = \prod_{t = 1}^T b_{s_t} (\mathbf{o})$
  * $P_{\lambda}(\mathbf{s}) = a_{s_0} \prod_{t = 2}^T a_{s_{t-1},s_t}$

In [1]:
using Distributions, LinearAlgebra, Combinatorics

# define Markov model's parameters (λ)
S = 2                                                  # number of states
a = [0.5; 0.5]                                         # initial state probability  
A = [0.7 0.3; 0.3 0.7]                                 # transmission Matrix  [a_11 a21; a12 a_22]
# Must be row stotastic: [sum(A[i,:]) for i in 1:size(A,1)] .== 1

B = [Categorical([0.9, 0.1]), Categorical([0.2, 0.8])] # observation distribution [b_1; b_2]
# Tells what is the probability of a state given an observation  P(O_t|S_i)
# e.g. `pdf(D[1],2) == 0.1` gives the probability of being in state 1 if we observed O_t = 2 

T = 5                     # time window length
o = [1;1;2;1;1]           # observations

s_all = collect(multiset_permutations([1;2],[5;5],5))

# this has all possible state sequences e.g. [1;1;1;1;1], [1;1;1;1;2] ...
# these are S^T permutations

# Probability of sequence of states
Pλ_s = zeros(length(s_all)) 
for z = 1:length(s_all)
    Pλ_s[z] = a[1] # initial distribution, since they are equal so we don't need to do this twice
    for t = 2:T
        Pλ_s[z] *= A[ s_all[z][t-1], s_all[z][t] ]
    end
end
@assert sum(Pλ_s) ≈ 1 # check it's a probability 

# Likelihood of O given a sequence of states
Lλ_o_s = ones(length(s_all)) 
for z = 1:length(s_all)
    for t = 1:T
        Lλ_o_s[z] *= pdf(B[ s_all[z][t] ], o[t])
    end
end
sum(Lλ_o_s)

sa = s_all[argmax(Pλ_s .* Lλ_o_s)]


┌ Info: Recompiling stale cache file /Users/nantonel/.julia/compiled/v1.2/Distributions/xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1240


5-element Array{Int64,1}:
 1
 1
 2
 1
 1

Obviously the brute force approach can easily become intractable as $S^T$. 

The forward algorithm can be used for this task: since   

$\alpha_j(t) = L_{\lambda} (o_1, \dots, o_t, s_t = j)$

is the likelihood being at a particular state given a set of observations 
we can infer the most probable state at a given time by

$ s^*(t) = \arg \max\{ \alpha_1 (t), \dots, \alpha_S(t) \}$


In [2]:
function Baum_forward(A,a,B,o)
    T, S = length(o), size(A,1)
    alpha = zeros(T,S)
    for j = 1:S
        alpha[1,j] += a[j]*pdf( B[j], o[1] )
    end
    
    for t = 2:T
        for j = 1:S
            for i = 1:S
                alpha[t,j] += alpha[t-1,i]*A[i,j]*pdf( B[j], o[t] )    
            end
        end
        # println(normalize(alpha[t,:],1)) # to check we get same results as wiki page
    end
    return alpha
end

alpha = Baum_forward(A,a,B,o)
sa2 = [argmax(alpha[t,:]) for t = 1:T]
@assert sa2 == sa


Viterbi computes the state sequence with two main differences with respect to the forward algorithm:

1. instead of the sum takes computes the _maximum_ of the previous paths
    * Forward algorithm: $\alpha_j(t) =  L_{\lambda} (o_1, \dots, o_t, s_t = j) = \sum_{i=1}^S \alpha_{t-1}(i) a_{ij} b_j(o_t)$
    * Viterbi algorithm $v_t (j) = \max_{j=1 \dots S} \{ v_{t-1}(i) \} a_{ij} b_j(o_t)$
    
2. keeps track of where these maxima occur, and backtrack the path


In [3]:
function viterbi(A,a,B,o)
    
    T, S = length(o), size(A,1)
    v = zeros(T,S)            # viterbi "probabilities"
    argmax_v = zeros(Int,T,S) # backtrack indices
  
    for s = 1:S
        v[1,s] = a[s]*pdf( B[s], o[1] )
    end
 
    # v[1,:] .= normalize(v[1,:],1)
    for t = 2:T       
        for s = 1:S

            argmax_vi = -1 
            vmax      = -Inf
            for ss = 1:S
                v_t = v[t-1,ss] * A[ss,s]  
                if v_t > vmax
                    argmax_vi = ss 
                    vmax = v_t
                end
            end
      
            v[t,s] = vmax * pdf( B[s], o[t] )
            argmax_v[t,s] = argmax_vi
        
        end
        #v[t,:] .= normalize(v[t,:],1)
    end

    bp = zeros(Int,T)
    bp[T] = argmax(v[end,:])
    for t in T-1:-1:1
        bp[t] = argmax_v[t+1,bp[t+1]]
    end
    return bp,v,argmax_v
end

bp,v,argmax_v = viterbi(A,a,B,o)
@assert sa == bp
v

5×2 Array{Float64,2}:
 0.45       0.1       
 0.2835     0.027     
 0.019845   0.06804   
 0.0183708  0.0095256 
 0.0115736  0.00133358

In [4]:
alpha

5×2 Array{Float64,2}:
 0.45       0.1       
 0.3105     0.041     
 0.022965   0.09748   
 0.0407876  0.0150251 
 0.0297529  0.00455077